## Build Database using weaviate-client


In [ ]:
!pip install weaviate-client


c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [2]:
import os
typhoon_api_key = os.getenv('TYPHOON_API_KEY')
weaviate_api_key = os.getenv('WEAVIATE_API_KEY')
weaviate_url = os.getenv('WEAVIATE_URL')
# print(typhoon_api_key)
# print(weaviate_api_key)

In [3]:
import weaviate
from datasets import load_dataset
import os

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
    # headers={"X-OpenAI-Api-Key": openai_api_key},
)


client.is_ready()

c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\weaviate\warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(


True

In [18]:
from weaviate.classes.config import Configure
questions = client.collections.create(
    name="Question",
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(), # Configure the Weaviate Embeddings integration
    generative_config=Configure.Generative.cohere()             # Configure the Cohere generative AI integration
)

In [ ]:
import weaviate
from weaviate.classes.init import Auth
import requests, json, os



resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

questions = client.collections.get("Question")

with questions.batch.dynamic() as batch:
    for d in data:
        batch.add_object({
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        })
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break

failed_objects = questions.batch.failed_objects
if failed_objects:
    print(f"Number of failed imports: {len(failed_objects)}")
    print(f"First failed object: {failed_objects[0]}")



c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\weaviate\warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(


In [20]:
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

In [22]:
import weaviate
from weaviate.classes.init import Auth
import os, json

questions = client.collections.get("Question")

response = questions.query.near_text(
    query="Thailand",
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))



{
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
}
{
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
}


## Quires Section using Typhoon API from PydanticAI

In [7]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

model = OpenAIModel(
    'typhoon-v2-70b-instruct',  # model library available at https://www.together.ai/models
    provider=OpenAIProvider(
        base_url='https://api.opentyphoon.ai/v1',
        api_key=typhoon_api_key,
    ),
)
agent = Agent(model)


In [5]:
import weaviate
from weaviate_agents.query import QueryAgent
import os

from pydantic_ai import Agent, RunContext
from typing import Optional, Dict, Any
import asyncio
from pydantic import BaseModel

In [9]:
# Define credential structure
class WeaviateCredentials(BaseModel):
    weaviate_url: str
    weaviate_api_key: str 
    typhoon_api_key: str
    collections: list[str]

class HelpfulResponse(BaseModel):
    helpful_response: str

model = OpenAIModel(
    'typhoon-v2-70b-instruct', 
    provider=OpenAIProvider(
        base_url='https://api.opentyphoon.ai/v1',
        api_key=typhoon_api_key,
    ),
)


agent = Agent(
    model=model,
    deps_type=WeaviateCredentials,
    result_type=HelpfulResponse,
    system_prompt=(
        "You are a helpful assistant that can search a knowledge base to answer questions. "
        "Use the search tool to find relevant information, then provide a comprehensive answer. "
        "If the search doesn't return useful information, acknowledge the limitations."
    ),
)

@agent.tool
def send_query_agent_request(ctx: RunContext[WeaviateCredentials], query: str) -> str:
    """
    Send a query to the Weaviate database and get the response.
    Using tool_plain which works better with synchronous functions.

    Args:
        ctx: The run context containing Weaviate credentials and configuration.
        query (str): The question or query to search for in the database.

    Returns:
        str: The response from the database containing relevant information.
    """
    try:
        # Get credentials from dependencies, with environment variables as fallback
        creds = ctx.deps
        weaviate_url = os.getenv("WEAVIATE_URL")
        weaviate_api_key = os.getenv("WEAVIATE_API_KEY")
        typhoon_api_key = os.getenv("TYPHOON_API_KEY")
        collections = creds.collections or ["Blogs"]
        
        if not all([weaviate_url, weaviate_api_key, typhoon_api_key, collections]):
            raise ValueError("Missing required credentials")
            
        # Connect to Weaviate Cloud instance
        weaviate_client = weaviate.connect_to_weaviate_cloud(
            cluster_url=weaviate_url,
            auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
            # headers={"X-OpenAI-Api-Key": openai_api_key},
        )
        
        # connect the query agent to your Weaviate collection(s)
        query_agent = QueryAgent(
            client=weaviate_client,
            collections=collections
        )
        
        result = query_agent.run(query)
        return result.final_answer
    except Exception as e:
        return f"Error querying Weaviate: {str(e)}"

In [26]:
response = client.collections.list_all(simple=False)

print(response)

{'Question': _CollectionConfig(name='Question', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.COHERE: 'generative-cohere'>, model={}), inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='answer', description="This property was generated by Weaviate's auto-schema feature on Sat Mar 22 10:45:15 2025", data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=False), vectorizer='text2vec-weaviate

In [27]:
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

In [28]:
custom_credentials = WeaviateCredentials(
    weaviate_url=os.getenv("WEAVIATE_URL"),
    weaviate_api_key=os.getenv("WEAVIATE_API_KEY"), 
    typhoon_api_key=os.getenv("TYPHOON_API_KEY"),
    collections=["Question"]
)


response = await agent.run(
    "What happened In 1953?",
    deps=custom_credentials
)
print(response.data.helpful_response)

c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\weaviate\warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In 1953, a significant scientific event was the discovery of the DNA double helix structure by James Watson and Francis Crick. They built the first accurate model of DNA's molecular structure, which is essential for carrying genetic information. This discovery marked a pivotal moment in the field of genetics and biology, paving the way for advances in understanding genetic coding and heredity.


In [10]:
custom_credentials = WeaviateCredentials(
    weaviate_url=os.getenv("WEAVIATE_URL"),
    weaviate_api_key=os.getenv("WEAVIATE_API_KEY"), 
    typhoon_api_key=os.getenv("TYPHOON_API_KEY"),
    collections=["Question"]
)


response = await agent.run(
    "What happened In 2025?",
    deps=custom_credentials
)
print(response.data.helpful_response)

c:\Users\natdhanai.pra\Documents\AI_LLMs\.venv\Lib\site-packages\weaviate\warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


I'm sorry, but I couldn't find any specific events scheduled for 2025 in the provided search results.


In [11]:
custom_credentials = WeaviateCredentials(
    weaviate_url=os.getenv("WEAVIATE_URL"),
    weaviate_api_key=os.getenv("WEAVIATE_API_KEY"), 
    typhoon_api_key=os.getenv("TYPHOON_API_KEY"),
    collections=["Question"]
)


response = await agent.run(
    "เกิดอะไรขึ้นในปี 1953?",
    deps=custom_credentials
)
print(response.data.helpful_response)

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=4360 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=3004 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


In 1953, a significant scientific milestone was achieved with the construction of the DNA model. James Watson and Francis Crick built a model of the molecular structure of DNA, which is the gene-carrying substance crucial to understanding genetics and hereditary processes.
